In [1]:
from keras.layers import Input, Lambda, Dense, Flatten
from keras.models import Model

from keras.preprocessing import image
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
import numpy as np
from glob import glob
import matplotlib.pyplot as plt



In [2]:
IMAGE_SIZE = [224, 224]

train_path = 'D:/thesis_word/images/training'
valid_path = 'D:/thesis_word/images/testing'

In [3]:
from tensorflow.keras.applications import MobileNetV2

mobilenetv2 = MobileNetV2(input_shape=IMAGE_SIZE + [3], weights='imagenet', include_top=False)

# don't train existing weights
for layer in mobilenetv2.layers:
    layer.trainable = False





9406464/9406464 [==============================] - 2s 0us/step


In [4]:
folders = glob('D:/thesis_word/images/training/*')

In [5]:
x = Flatten()(mobilenetv2 .output)

In [7]:
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Flatten, Dense
from tensorflow.keras.applications import MobileNetV2


mobilenetv2 = MobileNetV2(input_shape=IMAGE_SIZE + [3], weights='imagenet', include_top=False)


for layer in mobilenetv2.layers:
    layer.trainable = False


x = Flatten()(mobilenetv2.output)
prediction = Dense(1, activation='sigmoid')(x)

model = Model(inputs=mobilenetv2.input, outputs=prediction)

model.summary()

# Compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])


9406464/9406464 [==============================] - 0s 0us/step
Model: "model"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_2 (InputLayer)        [(None, 224, 224, 3)]        0         []                            
                                                                                                  
 Conv1 (Conv2D)              (None, 112, 112, 32)         864       ['input_2[0][0]']             
                                                                                                  
 bn_Conv1 (BatchNormalizati  (None, 112, 112, 32)         128       ['Conv1[0][0]']               
 on)                                                                                              
                                                                                                  
 Conv1_relu (ReLU)           (N

In [9]:
from keras.preprocessing.image import ImageDataGenerator

train_datagen = ImageDataGenerator(rescale = 1./255,
                                   shear_range = 0.2,
                                   zoom_range = 0.2,
                                   horizontal_flip = True)

test_datagen = ImageDataGenerator(rescale = 1./255)

training_set = train_datagen.flow_from_directory('/content/drive/MyDrive/Bone fracture_infoo/Dataset/canny Edge/training',
                                                 target_size = (224, 224),
                                                 batch_size = 32,
                                                 class_mode = 'categorical')

test_set = test_datagen.flow_from_directory('/content/drive/MyDrive/Bone fracture_infoo/Dataset/canny Edge/testing',
                                            target_size = (224, 224),
                                            batch_size = 32,
                                            class_mode = 'categorical')


Found 8845 images belonging to 3 classes.
Found 600 images belonging to 3 classes.


In [ ]:

r = model.fit(
    training_set,
    validation_data=test_set,
    epochs=100,
    steps_per_epoch=len(training_set),
    validation_steps=len(test_set)
)


plt.plot(r.history['loss'], label='train loss')
plt.plot(r.history['val_loss'], label='val loss')
plt.legend()
plt.show()
plt.savefig('LossVal_loss')

# Accuracies
plt.plot(r.history['accuracy'], label='train acc')
plt.plot(r.history['val_accuracy'], label='val acc')
plt.legend()
plt.show()
plt.savefig('AccVal_acc')


from sklearn.metrics import confusion_matrix
import numpy as np

# Assuming you have test data and ground truth labels
y_true = np.concatenate([test_set[i][1] for i in range(len(test_set))])
y_pred_probs = model.predict(test_set)
y_pred = np.argmax(y_pred_probs, axis=1)


cm = confusion_matrix(np.argmax(y_true, axis=1), y_pred)

print("Confusion Matrix:")
print(cm)


Epoch 1/100
 45/277 [===>..........................] - ETA: 27:49 - loss: 1.2931 - accuracy: 0.5653

In [ ]:
# Assuming you have test data and ground truth labels
y_true = np.concatenate([test_set[i][1] for i in range(len(test_set))])
y_pred_probs = model.predict(test_set)
y_pred = np.argmax(y_pred_probs, axis=1)

cm = confusion_matrix(np.argmax(y_true, axis=1), y_pred)

# Plot confusion matrix as a heatmap
plt.figure(figsize=(10, 8))
sns.set(font_scale=1.4)  # Adjust font size for better readability
sns.heatmap(cm, annot=True, fmt='g', cmap='Blues', cbar=False)
plt.xlabel('Predicted labels')
plt.ylabel('True labels')
plt.title('Confusion Matrix')
plt.savefig('Confusion_Matrix.png')  # Save the confusion matrix as a PNG file
plt.show()

In [ ]:
model_path = 'model/my_model.h5'

# Save the model in .h5 format
tf.keras.models.save_model(model, model_path, save_format='h5')